In [1]:
using Optim
using Random
using StatsBase
using Statistics
using Distributions
using LinearAlgebra

Inicialmente, vamos elaborar funções para gerar jogadores aleatórios e simular campeonatos, além de outra para, dados os resultados, calcular a verossimilhança dos parâmetros em teste (função que deve ser maximizada).

```create_players``` recebe como parâmetros a quantidade de clubes e jogadores em cada clube e cria jogadores aleatórios.

```simulating_game``` recebe dois clubes e uma quantidade de simulações e simula os jogos entre esses dois clubes por meio de uma Poisson com média $\lambda = \frac{\lambda_a}{\lambda_d}$, onde $\lambda_a$ é o somatório das forças dos jogadores do clube atacante ($\mathcal{C}_a$) e $\lambda_d$ é o somatório das forças dos jogadores do clube defensor ($\mathcal{C}_d$).

```create_games``` recebe a quantidade de temporadas, clubes e jogadores por clube e cria jogos entre todos os clubes simulando essas temporadas.

```likelihood``` recebe os resultados e um conjunto de parâmetros e retorna a log verossimilhança negativa desses parâmetros com o resultado.

```gradient``` gradiente da função likelihood.

As duas últimas funções podem ser melhores explicadas matematicamente, dessa forma, como explicado anteriormente, cada placar é composto por duas variáveis aleatórias vindas de uma Poisson com média igual a razão das somas das forças dos dois clubes, assim seja $k$ a quantidade de gols de um clube contra outro, $\lambda_a$ a soma das forças do clube que marcou os gols e $\lambda_d$ do clube que sofreu. Dessa forma, a verossimilhança desse pedaço de placar é dada por $P(X = k | proficiências) = \frac{\lambda^k\exp{-\lambda}}{k!}$, onde $\lambda = \frac{\lambda_a}{\lambda_d}$. Note que a verossimilhança total é o produto de todos os "pedaços" de placar, mas, como cada fator desses é um valor em $[0, 1]$, isso se torna muito pequeno, podendo ocorrer underflow error, por isso opta-se a usar o log da verossimilhança (como o log é uma função estritamente crescente, a comparação entre os resultados é mantida). Por fim, os fatores se tornam parcelas, agora todas negativas, então multiplicamos por $-1$, transformando essa métrica num valor positivo e, mais importante, mudando o problema para um problema de minimização.

Agora, como citado no parágrafo anterior, cada parcela da verossimilhança é dada pela expressão $-\log{\frac{\lambda^k\exp{-\lambda}}{k!}}$, dessa forma, para cada jogador $j$, temos que a derivada dessa parcela, em relação ao jogador $j$, será:
  - $0$, se $j \notin \mathcal{C}_a \land j \notin \mathcal{C}_d$;
  - $\dfrac{\lambda_d - \lambda_a}{\lambda_d^2} + k \left(\dfrac{1}{\lambda_d} - \dfrac{1}{\lambda_a}\right)$, se $j \in \mathcal{C}_a \land j \in \mathcal{C}_d$;
  - $\dfrac{1}{\lambda_d} - \dfrac{k}{\lambda_a}$, se $j \in \mathcal{C}_a \land j \notin \mathcal{C}_d$ e;
  - $-\dfrac{\lambda_a}{\lambda_d^2} + \dfrac{k}{\lambda_d}$, se $j \notin \mathcal{C}_a \land j \in \mathcal{C}_d$.
  
Assim, passando por cada jogo e fazendo as derivadas parciais em relação a cada jogador, chegamos ao gradiente da função de verossimilhança.

In [2]:
function creat_players(clubs, players_per_club, mean = 0, var = 1, lb = 0, ub = 100)
    all_players = rand(Truncated(Normal(mean, var), lb, ub), clubs * players_per_club)
    all_players = all_players / (sum(all_players) / clubs)
    players = Dict{Int64, Float64}(zip(1:clubs * players_per_club, all_players))
    return players
end

function simulating_game(club1, club2, sims = 1000000)
    λ₁ = sum(club1)
    λ₂ = sum(club2)
    X₁ = Poisson(λ₁ / λ₂)
    X₂ = Poisson(λ₂ / λ₁)
    Y₁ = rand(X₁, sims)
    Y₂ = rand(X₂, sims)
    wins₁ = sum(Y₁ .> Y₂)
    draws = sum(Y₁ .== Y₂)
    wins₂ = sum(Y₁ .< Y₂)
    return wins₁, draws, wins₂
end

function create_games(seasons, clubs, ppc)
    results = [[[0 for i in 1:4] for j in 1:clubs * (clubs - 1)] for s in 1:seasons]
    players = creat_players(clubs, ppc)
    squads = []
    for i in 1:seasons
        line = 1
        append!(squads, [reshape(shuffle(collect(1:length(players))), (clubs, ppc))])
        actual_clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
        for i in 1:clubs
            for j in 1:ppc
                actual_clubs[i, j] = players[actual_clubs[i, j]]
            end
        end

        for j in 1:clubs
            for k in 1:clubs
                if j != k
                    Xⱼ = Poisson(sum(actual_clubs[j, :]) / sum(actual_clubs[k, :]))
                    Xₖ = Poisson(sum(actual_clubs[k, :]) / sum(actual_clubs[j, :]))
                    results[i][line][1] = j
                    results[i][line][2] = rand(Xⱼ)
                    results[i][line][3] = rand(Xₖ)
                    results[i][line][4] = k
                    line += 1
                end
            end
        end
    end
    return results, squads, players
end

# results be an array like [1, 1, 0, 3]
# [home club, score home, score away, away club]
function likelihood(players, results, squads)
    if typeof(players) != Dict{Int64, Float64}
        players = Dict{Int64, Float64}(zip(1:length(players), players))
    end
    
    loglikelihood = 0
    for i in 1:length(results)
        actual_clubs = convert(Matrix{Float64}, (deepcopy(squads[i])))
        for j in 1:size(actual_clubs)[1]
            for k in 1:size(actual_clubs)[2]
                actual_clubs[j, k] = players[actual_clubs[j, k]]
            end
        end
        
        for j in 1:length(results[i])
            clubₕ, scoreₕ, scoreₐ, clubₐ = results[i][j]
            loglikelihood -= logpdf(Poisson(sum(actual_clubs[clubₕ, :]) / sum(actual_clubs[clubₐ, :])),
                                    scoreₕ)
            loglikelihood -= logpdf(Poisson(sum(actual_clubs[clubₐ, :]) / sum(actual_clubs[clubₕ, :])),
                                    scoreₐ)
        end
    end
    
    return loglikelihood
end

function gradient(players, results, squads, ∇likelihood)
    if typeof(players) != Dict{Int64, Float64}
        players = Dict{Int64, Float64}(zip(1:length(players), players))
    end
    
    # ∇likelihood = zeros(length(players))
    for i in 1:length(results)
        for j in 1:length(results[i])
            clubₕ, scoreₕ, scoreₐ, clubₐ = results[i][j]
            λₕ, λₐ = 0, 0
            for k in 1:size(squads[i], 2)
                λₕ += players[squads[i][clubₕ, k]]
                λₐ += players[squads[i][clubₐ, k]]
            end
            
            for p in 1:length(players)
                if p in squads[i][clubₕ, :] && p in squads[i][clubₐ, :]
                    ∇likelihood[p] -= (λₕ - λₐ) / λₐ ^ 2 + scoreₕ * (λₐ - λₕ) / (λₕ * λₐ)
                    ∇likelihood[p] -= (λₐ - λₕ) / λₕ ^ 2 + scoreₐ * (λₕ - λₐ) / (λₐ * λₕ)
                elseif p in squads[i][clubₕ, :]
                    ∇likelihood[p] -= scoreₕ / λₕ - 1 / λₐ
                    ∇likelihood[p] -= λₐ / λₕ ^ 2 - scoreₐ / λₕ
                elseif p in squads[i][clubₐ, :]
                    ∇likelihood[p] -= scoreₐ / λₐ - 1 / λₕ
                    ∇likelihood[p] -= λₕ / λₐ ^ 2 - scoreₕ / λₐ
                end
            end
        end
    end
    
    return ∇likelihood
end

gradient (generic function with 1 method)

In [3]:
seasons = 10
n_clubs = 20
ppc = 11

# compilando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(players, results, squads)
@time gradient(players, results, squads, zeros(length(players)))

# reexecutando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(players, results, squads)
@time gradient(players, results, squads, zeros(length(players)))
println()

  1.277383 seconds (2.14 M allocations: 126.719 MiB, 8.04% gc time, 99.04% compilation time)
  0.529174 seconds (2.55 M allocations: 146.259 MiB, 9.64% gc time, 98.17% compilation time)
  1.501303 seconds (5.30 M allocations: 470.564 MiB, 3.98% gc time, 13.25% compilation time)
  0.012657 seconds (69.11 k allocations: 3.847 MiB)
  0.009642 seconds (83.26 k allocations: 3.674 MiB)
  1.257706 seconds (5.15 M allocations: 461.898 MiB, 2.81% gc time)



# Teste para Otimização

In [4]:
f(x) = likelihood(x, results, squads)
g(∇likelihood, x) = gradient(x, results, squads, ∇likelihood)
lower = zeros(220)
upper = 20 * ones(220)
x_inicial = rand(220)
od = OnceDifferentiable(f, g, x_inicial)
@time res1  = optimize(od,
                       lower,
                       upper,
                       x_inicial,
                       Fminbox(GradientDescent()),
                       Optim.Options(iterations = 1000))
@time res2  = optimize(od,
                       lower,
                       upper,
                       x_inicial,
                       Fminbox(NelderMead()),
                       Optim.Options(iterations = 1000))
@time res3 = optimize(x -> likelihood(x, results, squads),
                      lower,
                      upper,
                      x_inicial,
                      Fminbox(NelderMead()),
                      Optim.Options(iterations = 1000))

println()

  6.343651 seconds (22.30 M allocations: 1.505 GiB, 5.65% gc time, 57.77% compilation time)
  6.074482 seconds (38.97 M allocations: 1.707 GiB, 5.98% gc time, 23.38% compilation time)
1977.286300 seconds (16.55 G allocations: 715.159 GiB, 2.09% gc time, 0.03% compilation time)



In [5]:
Optim.converged(res1), Optim.minimizer(res1), Optim.minimum(res1), likelihood(players, results, squads)

(true, [0.9228364351467517, 0.024655827036760947, 0.4073093411623929, 0.19800638547167937, 0.33810687578918497, 0.32851317929292567, 0.3022988256810326, 0.6661517492376108, 0.22016145224289896, 0.3572206232105273  …  0.18706936055570877, 0.11963943833656443, 0.5859548517486248, 0.24741976261254295, 0.6916316639711744, 0.7762796582546487, 0.4878498233613584, 0.7629648794759596, 0.2302181139397641, 0.7668839088447497], 10631.156304147828, 9867.14341883495)

In [6]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res1)), Optim.minimizer(res1)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res1), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

Correlação de Pearson: 0.02522847938352497
Correlação de Spearman: 0.02426872080377318
Correlação de Kendall: 0.014943960149439602
||Gradiente||: 394.72884677342324


In [7]:
Optim.converged(res2), Optim.minimizer(res2), Optim.minimum(res2), likelihood(players, results, squads)

(true, [0.9228364351467517, 0.024655827036760947, 0.4073093411623929, 0.19800638547167937, 0.33810687578918497, 0.32851317929292567, 0.3022988256810326, 0.6661517492376108, 0.22016145224289896, 0.3572206232105273  …  0.18706936055570877, 0.11963943833656443, 0.5859548517486248, 0.24741976261254295, 0.6916316639711744, 0.7762796582546487, 0.4878498233613584, 0.7629648794759596, 0.2302181139397641, 0.7668839088447497], 10604.759765940285, 9867.14341883495)

In [8]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res2)), Optim.minimizer(res2)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res2), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

Correlação de Pearson: 0.03282224878142021
Correlação de Spearman: 0.03144542806106061
Correlação de Kendall: 0.02050643420506434
||Gradiente||: 360.4287343613268


In [9]:
Optim.converged(res3), Optim.minimizer(res3), Optim.minimum(res3), likelihood(players, results, squads)

(true, [0.5837995817526539, 0.8396186565867251, 0.02351462085244894, 0.15402068438060315, 0.581483194568835, 1.088115826910971, 0.3013235722292648, 0.04292083931089695, 0.36685111241916524, 0.07619085824580343  …  1.8877360155311786, 0.48695819364269577, 0.09480834749069246, 0.2612906199547271, 0.29137662598194475, 0.4188285061822444, 0.3809249749730349, 0.26142581999849346, 0.9040687871089926, 0.32249017661667617], 9789.582572915311, 9867.14341883495)

In [10]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res3)), Optim.minimizer(res3)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res3), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

Correlação de Pearson: 0.7658380399762424
Correlação de Spearman: 0.671018747569916
Correlação de Kendall: 0.4839352428393524
||Gradiente||: 5.25691156129967


In [11]:
@time res4  = optimize(od,
                       lower,
                       upper,
                       Optim.minimizer(res3),
                       Fminbox(GradientDescent()),
                       Optim.Options(iterations = 1000))

res4

  2.519544 seconds (10.54 M allocations: 934.922 MiB, 1.93% gc time)


 * Status: success

 * Candidate solution
    Final objective value:     9.789583e+03

 * Found with
    Algorithm:     Fminbox with Gradient Descent

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   3  (vs limit Inf)
    Iterations:    1
    f(x) calls:    2
    ∇f(x) calls:   2


In [12]:
Optim.minimizer(res3) == Optim.minimizer(res4)

true